참고문헌 : Pytel 외 저, 이주성 외 역, 재료역학, 2판, 한티미디어, 2013.<br>Ref: Pytel, Kiusalaas, Sharma, Mechanics of Materials, 2nd Ed., Cengage Learning, 2013.

`python` 기능을 확장해 주는 `module`을 불러 들임 (일부 기능만 사용될 수도 있음)<br>
Bring in `module`'s that would expand features of `python`. (This file may use just some of them.)

In [ ]:
import numpy as np  # 배열, 행렬 관련 기능
import numpy.linalg as na  # 선형대수 (벡터, 행렬) 관련 기능
import matplotlib.pyplot as plt  # 그래프 관련 기능
import scipy.integrate as si  # 적분 관련 기능
import sympy as sy  # 기호 연산 기능
import sympy.plotting as splot
import IPython.display as disp  # 웹페이지 표시 기능
sy.init_printing()  # 기호 연산 결과 표시 기능 준비

## 예제 07.001<br>ex07.001

부정정보: 이중적분<br>Statically Indeterminate Beam : Double integration

p. 296

### 문제에서 주어진 변수<br>Given Parameters

#### 보의 길이<br>Length of the beam

In [ ]:
L_AB_m = sy.symbols('L_{AB}[m]', real=True, nonnegative=True)

s_d = {
    L_AB_m: 12,
}


#### 하중<br>Load

In [ ]:
w_AB_N_m = sy.symbols('w_{AB}[N/m]', real=True)

s_d.update(
    {
        w_AB_N_m: -600,
    }
)


보를 그려주는 모듈을 `import` 할 수 있도록 준비.

In [ ]:
import os   # 운영체제 관련 기능 Operating Systems
import sys  # 시스템 관련 기능 Systems
# utils 폴더의 모듈을 import 할 수 있도록 준비
# add utils folder to sys.path to import
sys.path.append(os.path.abspath(os.path.join(os.pardir, 'utils')))
# 선도 관련 기능 diagrams
import draw_diagrams

points_list = [
    {'x_m': 0, 'text':'A'},
    {'x_m': L_AB_m.subs(s_d), 'text':'B'},
]

reaction_list = [
    {'x_m': 0},
    {'x_m': L_AB_m.subs(s_d)},
]

dist_load_list = [
    {'x_begin_m': 0, 'x_end_m': L_AB_m.subs(s_d), 'text': 'w0'}
]

moment_list = [
    {'x_m': 0, 'direction': 'ccw', 'text': 'M', 'open':'right'},
]

draw_diagrams.draw_beam(L_AB_m.subs(s_d), points_list, reaction_list, dist_load_list=dist_load_list, moment_list=moment_list)

In [ ]:
x_m = sy.symbols('x[m]', nonnegative=True)
x_A_m = 0
x_B_m = L_AB_m


### 반력<br>Reaction forces

In [ ]:
R_A_N, R_B_N, RM_A_Nm = sy.symbols('R_{A}[N], R_{B}[N], RM_{A}[Nm]', real=True)

### 평형방정식<br>Equilibrium Equations

In [ ]:
fy_eq = sy.Eq(R_A_N + R_B_N + w_AB_N_m * L_AB_m)

In [ ]:
fy_eq

In [ ]:
m_eq = sy.Eq(RM_A_Nm + R_B_N * L_AB_m + sy.integrate(w_AB_N_m * x_m, (x_m, x_A_m, x_B_m)))

In [ ]:
m_eq

### 굽힘모멘트선도 <br>Bending moment diagram

In [ ]:
M = - RM_A_Nm + R_A_N * x_m + sy.Rational(1, 2) * w_AB_N_m * x_m ** 2

In [ ]:
M

### 처짐<br>Deflection

In [ ]:
EInu_Nm3 = sy.Function('EI\\nu_Nm3')
# http://docs.sympy.org/latest/modules/solvers/ode.html#dsolve
# http://docs.sympy.org/latest/tutorial/calculus.html
EInu_pp_Nm = sy.Derivative(EInu_Nm3(x_m), x_m, x_m)
defl_deq = sy.Eq(EInu_pp_Nm, M)

In [ ]:
defl_deq

In [ ]:
EInu_Nm3_sol_eq = sy.dsolve(defl_deq, EInu_Nm3(x_m))

In [ ]:
EInu_Nm3_sol = EInu_Nm3_sol_eq.rhs

In [ ]:
EInu_Nm3_sol

### 경계조건<br>Boundary conditions

$$\nu(x=0)=0$$

In [ ]:
nu_0_eq = sy.Eq(EInu_Nm3_sol.subs(x_m, 0), 0)

In [ ]:
nu_0_eq

$$\theta(x=0)=0$$

In [ ]:
theta_0_eq = sy.Eq(sy.diff(EInu_Nm3_sol, x_m).subs(x_m, 0), 0)

In [ ]:
theta_0_eq

$$\nu(x=L)=0$$

In [ ]:
nu_L_eq = sy.Eq(EInu_Nm3_sol.subs(x_m, x_B_m), 0)

In [ ]:
nu_L_eq

연립방정식을 풂<br>Solve for the simultaneous equations

In [ ]:
const_sol = sy.solve((nu_0_eq, theta_0_eq, nu_L_eq), (nu_0_eq.lhs, theta_0_eq.lhs, R_A_N))

In [ ]:
const_sol

In [ ]:
EInu_Nm3

### 평형방정식<br>Equilibrium equations

In [ ]:
fy_eq.subs(const_sol)

In [ ]:
m_eq.subs(const_sol)

In [ ]:
residual_eq = sy.Eq(R_A_N, const_sol[R_A_N])

In [ ]:
rf_sol = sy.solve((fy_eq.subs(const_sol), m_eq.subs(const_sol), residual_eq), 
                  (R_A_N, R_B_N, RM_A_Nm))

모든 반력의 해<br>Solutions of all reaction forces

In [ ]:
rf_sol

In [ ]:
rf_sol[R_A_N].subs(s_d)

In [ ]:
rf_sol[R_B_N].subs(s_d)

In [ ]:
rf_sol[RM_A_Nm].subs(s_d)

### 처짐<br>Deflection

In [ ]:
EInu_Nm3_sol.subs(const_sol).subs(rf_sol)

In [ ]:
splot.plot(EInu_Nm3_sol.subs(const_sol).subs(rf_sol).subs(s_d), (x_m, 0, L_AB_m.subs(s_d)), ylabel='$EI\\nu(Nm^3)$', xlabel='x(m)')